# WINDNODE ABW - Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__author__ 		= "Jonathan Amme" <br>

GENERAL TODO
- [ ] use consistent scenario order, cf. https://github.com/windnode/WindNODE_ABW/issues/64#issuecomment-670712969

# Table of Contents

* [0 Key Results](#0_key_results)
* [1 Demand and Generation](#1_demand_and_generation)
* [2 Area required by RES](#2_area_required_by_res)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
} 

In [ ]:
####### WINDNODE ########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()

# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')

from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######### DATA ##########

import re
import pandas as pd

####### Plotting ########

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
# set seaborn style
sns.set()

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
# papermill parameters will be inserted below this tagged cell

TODO
- [ ] suppress long log output of analysis

In [ ]:
# obtain processed results
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=scenarios,
                                      force_new_results=force_new_results)

In [ ]:
##### Usefull stuff ######
scn = list(scenarios)[-2]
# Names of Municialities
MUN_NAMES = regions_scns[scn].muns.gen.to_dict()
# extend for total ABW region
MUN_NAMES.update({100:'ABW'})
# Colormap
CMAP = px.colors.sequential.GnBu_r
#CMAP_NAME_KEY_PLOT = 'GnBu_r'
CMAP_NAME_KEY_PLOT = 'viridis'

# 0 Key Results <a class="anchor" id="0_key_results"></a>

The following plots provide an overview on the key results of all scenarios.

TODO
- [ ] add explanations on params
- [ ] rework autarky as soon as #135 is merged
- [ ] fix title position

In [ ]:
scenario_results = plot_key_scenario_results(results_scns, scenarios, CMAP_NAME_KEY_PLOT)

# 1 Demand and Generation <a class="anchor" id="1_demand_and_generation"></a>

## 1.1 Installed Capacities Electricity/Heat

TODO
- [ ] Fix title overlap
- [ ] Scenario order?
- [ ] Fix Autarky as soon as PR 135 is done

In [ ]:
cap_el = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity electricity supply'].sum(axis=0)
                       for scn in scenarios}).T

cap_el = cap_el.assign(sum=cap_el.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1) \
    .rename(columns=PRINT_NAMES)

fig = go.Figure()

for tech, data in cap_el.iteritems():
    fig.add_trace(go.Bar(x=cap_el.index,
                         y=data,
                         name=tech,
                         #orientation='h',
                         showlegend=True))

fig.update_layout(
    title='Installed Capacity Electricity Supply',
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis_tickfont_size=12,
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)

fig.show()

In [ ]:
cap_th = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity heat supply'].sum(axis=0)
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

cap_th = cap_th.assign(sum=cap_th.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1) \
    .rename(columns=PRINT_NAMES)

fig = go.Figure()

for tech, data in cap_th.iteritems():
    fig.add_trace(go.Bar(x=cap_th.index,
                         y=data,
                         name=tech,
                         #orientation='h',
                         showlegend=True))

fig.update_layout(
    title='Installed Capacity Heat Supply',
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis_tickfont_size=12,
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

## 1.2 Electricity and Heat Generation

In [ ]:
gen_el = pd.DataFrame({scn: results_scns[scn]['results_t']['Electricity generation']
                       for scn in scenarios}).T
gen_th = pd.DataFrame({scn: results_scns[scn]['results_t']['Heat generation']
                       for scn in scenarios}).T
df_gen = pd.concat([gen_el, gen_th], axis=0, keys=['Electricity','Heat'])
df_gen = df_gen / 1e3
df_gen = df_gen.rename(columns=PRINT_NAMES)
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.05, subplot_titles=['Electricity','Heat'])
for sector, df in df_gen.groupby(level=0):
    row = 1 if sector == 'Electricity' else 2

    for tech, data in df.loc[sector].iteritems():
        fig.add_trace(go.Bar(y=data.index,
                             x=data,
                             name=tech,
                             orientation='h',
                             hovertemplate='%{x:.1f} GWh',
                             showlegend=True),row=row, col=1)
fig.update_layout(
    title='Generation',
    barmode='stack',
    hovermode="y unified",
    height=50*len(scenarios),
    autosize=True)

fig.update_xaxes(title_text="GWH", row=2, col=1)
fig.show()

In [ ]:
gen_el = pd.DataFrame({scn: results_scns[scn]['results_t']['Electricity generation']
                       for scn in scenarios}).T
gen_th = pd.DataFrame({scn: results_scns[scn]['results_t']['Heat generation']
                       for scn in scenarios}).T
df_gen = pd.concat([gen_el, gen_th], axis=0, keys=['Electricity','Heat'])
df_gen = df_gen.rename(columns=PRINT_NAMES)
df_gen = df_gen / 1e3

fig = make_subplots(rows=len(scenarios), cols=1, shared_xaxes=True, vertical_spacing=0)
for row, (key, df) in enumerate(df_gen.groupby(level=1)):
    for i, (tech, data) in enumerate(df.iteritems()):
        fig.add_trace(go.Bar(y=list(zip(*data.index.swaplevel())),
                             x=data,
                             name=tech,
                             orientation='h',
                             #marker_color=CMAP[i+1],
                             legendgroup=tech,
                             hovertemplate='%{x:.1f} GWh',
                             showlegend=not bool(row)), row=row+1, col=1)

fig.update_layout(
    title='Demand',
    barmode='stack',
    hovermode="y unified",
    height=50*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_xaxes(title_text="GWH", row=len(scenarios), col=1, matches='x')
fig.show() 

## 1.3 Electricity and Heat Demand

In [ ]:
df_el = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Stromnachfrage nach Gemeinde'].sum()
                       for scn in scenarios}).T
df_heat = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Wärmenachfrage nach Gemeinde'].sum()
                       for scn in scenarios}).T
df_heat = df_heat.T.groupby([s.split('_')[0] for s in df_heat.columns.values]).sum().T

df_demand = pd.concat([df_el, df_heat], axis=0, keys=['Electricity', 'Heat'])
df_demand = df_demand.rename(columns=PRINT_NAMES)
df_demand = df_demand / 1e3

fig = make_subplots(rows=len(scenarios), cols=1, shared_xaxes=True, vertical_spacing=0)
for row, (key, df) in enumerate(df_demand.groupby(level=1)):
    for i, (tech, data) in enumerate(df.iteritems()):
        fig.add_trace(go.Bar(y=list(zip(*data.index.swaplevel())),
                             x=data,
                             name=tech,
                             orientation='h',
                             marker_color=CMAP[i+1],
                             legendgroup=tech,
                             hovertemplate='%{x:.1f} GWh',
                             showlegend=not bool(row)), row=row+1, col=1)

fig.update_layout(
    title='Demand',
    barmode='stack',
    hovermode="y unified",
    height=50*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_xaxes(title_text="GWH", row=len(scenarios), col=1, matches='x')
fig.show() 

# 2 Area required by RES <a class="anchor" id="2_area_required_by_res"></a>

### false plots / templates

In [ ]:
df_area = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Area required'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

fig = go.Figure()
for i, (tech, data) in enumerate(df_area.iteritems()):
    fig.add_trace(go.Bar(y=df_area.index,
                         x=data,
                         name=tech,
                         orientation='h',
                         marker_color=CMAP[i+1],
                         hovertemplate='%{x:.1f} ha',
                         showlegend=True))

fig.update_layout(
    title='Required Area',
    barmode='group',
    hovermode="y unified",
    height=50*len(scenarios),
    autosize=True)
fig.update_xaxes(title_text="ha")
fig.show()

In [ ]:
df_data = results_scns[list(scenarios)[0]]['highlevel_results']
traces = ['PV rooftop', 'PV ground', 'rel. Wind']

fig = go.Figure()

for i, trace in enumerate(traces):
    mask = [i for i in df_data.index if trace in  i[0]]
    data = df_data.loc[mask]
    index = data.index.get_level_values(level=0)

    fig.add_trace(
        go.Bar(y=index, x=data.values,
               orientation='h',
               name=trace,
               marker_color=CMAP[2*i+1],
              hovertemplate='%{x:.1f} %'))
    
fig.update_layout(title_text = 'Relative Required Area',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)
fig.update_xaxes(showspikes=True)

fig.show()

## 2.1 Available Areas: Installable Power

The following plot shows the maximum installable power for each land use scenario for each technology as absolute values (colored) and relative to the max. installable power for the regulatory status quo (grey). The regulatory status quo is shown in bold for each technology.

Note: The specific land use (ha/MWp) of PV are different for 2035 and 2050. For the sake of simplicity, the value of 2050 is used in this plot (cf. assumptions).

In [ ]:
# get potential areas per land use scenario
pv_ground = pd.Series({f'{sc}_{faktor}':regions_scns['ISE'].pot_areas_pv_scn(
    scenario=sc, pv_usable_area_agri_max=2086*faktor)['with_agri_restrictions'].sum()
                       for sc in ['HS', 'H'] for faktor in [0.1, 1, 2]})
wind = pd.Series({sc:regions_scns['ISE'].pot_areas_wec_scn(
    scenario=sc).sum()*faktor for sc, faktor in zip(['SQ', 's1000f1', 's500f0', 's500f1'],
                                                    [1, 0.1, 0.1, 0.1])})
pv_roof_tmp = regions_scns['ISE'].pot_areas_pv_roof.sum(axis=0)
re_pot_config = regions_scns['ISE'].cfg['scn_data']['generation']['re_potentials']
pv_roof = pd.Series(
    {'roof':  pv_roof_tmp['area_resid_ha'] * re_pot_config['pv_roof_resid_usable_area'] + 
              pv_roof_tmp['area_ind_ha'] * re_pot_config['pv_roof_ind_usable_area'] }
)

df_pot_area = pd.concat([pv_ground, pv_roof, wind] , keys=['pv_ground', 'pv_roof', 'wind'])

# Land use
df_area = pd.DataFrame({scn: regions_scns[scn].cfg['scn_data']['generation']['re_potentials']
                       for scn in scenarios}).T
df_land_use = pd.concat([
    df_area['pv_roof_land_use'], # ha/Mwp
    df_area['pv_land_use'].rename('pv_ground_land_use'), # ha/MWp
    (df_area['wec_land_use'] / df_area['wec_nom_power']).rename('wind_land_use')],  # ha /MW
    axis=1)

highlight_dict = {
    'wind': {'Wind legal SQ \(VR/EG\)':
             '<b>Wind legal SQ (VR/EG)</b>'},
    'pv_ground': {'PV ground HS 0.1-perc agri':
                  '<b>PV ground HS 0.1-perc agri</b>'},
    'pv_roof': {'PV rooftop':
                '<b>PV rooftop</b>'}
}
PRINT_NAMES_WO_REL = {k:v.replace('Area required rel. ', '') for k, v in PRINT_NAMES.items()}

fig = make_subplots(specs=[[{"secondary_y": True}]])

CMAP2 = px.colors.sequential.Greys

# potential Capacity
for i, (tech, df) in enumerate(df_pot_area.groupby(level=0)):
    
    data = df.loc[tech] / df_land_use.loc['ISE', tech+'_land_use']
    data = data.rename(index=PRINT_NAMES_WO_REL)
    data.index = data.index.str.replace(list(highlight_dict[tech].keys())[0],
                                        list(highlight_dict[tech].values())[0])
    
    fig.add_trace(
        go.Bar(x=data.index,
               y=data / 1e3,
               orientation='v',
               name=PRINT_NAMES_WO_REL[tech],
               marker_color=CMAP[2*i+1],
               legendgroup=tech,
               opacity=1,
               offsetgroup=1,
               hovertemplate='%{y:.1f} GW'))
    
    
    data = df.loc[tech] / df.loc[(tech,['HS_0.1', 'roof', 'SQ'])].values * 100
    data = data.rename(index=PRINT_NAMES_WO_REL)
    data.index = data.index.str.replace(list(highlight_dict[tech].keys())[0],
                                        list(highlight_dict[tech].values())[0])
    
    fig.add_trace(
        go.Bar(x=data.index,
               y=data,
               orientation='v',
               name='rel. to legal SQ',
               marker_color='grey',
               legendgroup=tech,
               showlegend=True,
               opacity=0.4,
               offsetgroup=2,
               hovertemplate='%{y:.1f} %'),
        secondary_y=True)
    
fig.update_layout(title_text = 'RES Power Potential in Land Use Scenarios',
                    autosize=True,
                  barmode='group',
                 hovermode="x unified")

fig.update_yaxes(title="GW", showspikes=True, secondary_y=False)
fig.update_yaxes(title="%", showspikes=True, secondary_y=True)

fig.show()

### 2.1.1 Available areas: Wind Energy

### 2.1.2 Available areas: Ground-Mounted PV

### 2.1.3 Available areas: Roof-Mounted PV

## 2.2 Area usage

# 4. Energy Mix

### 4.1 Electricity Generation and Demand

In [ ]:
df_generation = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Stromerzeugung'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)
df_demand = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Stromnachfrage'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

fig = go.Figure()
for tech, data in df_generation.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=data / 1e3,
                            orientation='v',
                            name=tech,
#                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh'))

for tech, data in df_demand.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=-data / 1e3,
                            orientation='v',
                            name=tech,
#                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh',
#                             visible='legendonly'
                        ))


fig.update_layout(
    title='Power Generation and Demand',
    barmode='relative',
#     height=600,
    autosize=True,
    hovermode="x unified")
fig.update_yaxes(title='GWh', showspikes=False)
fig.show()

## 4.2 Heat Generation and Demand

In [ ]:
df_generation = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Wärmeerzeugung'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)
df_demand = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Wärmenachfrage'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

fig = go.Figure()
for tech, data in df_generation.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=data / 1e3,
                            orientation='v',
                            name=tech,
#                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh'))

for tech, data in df_demand.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=-data / 1e3,
                            orientation='v',
                            name=tech,
#                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh',
#                             visible='legendonly'
                        ))


fig.update_layout(
    title='Heat Generation and Demand',
    barmode='relative',
#     height=600,
    autosize=True,
    hovermode="x unified")
fig.update_yaxes(title='GWh', showspikes=False)
fig.show()

# 5 Emissions

## 5.1 Emissions absolute

In [ ]:
df_emissions = pd.DataFrame(
    {scn : pd.concat([pd.Series(data=results_scns[scn]['results_axlxt'][key].sum().sum(), index=[key]) \
                      for key in results_scns[scn]['results_axlxt'] if 'total' in key]) \
     for scn in scenarios}).T

df_emissions['CO2 emissions grid total'] = df_emissions[['CO2 emissions grid total',
                                                         'CO2 emissions grid new total']]
df_emissions = df_emissions.drop(columns='CO2 emissions grid new total')

fig = go.Figure()
for i, (tech, data) in enumerate(df_emissions.iteritems()):
    fig.add_trace(go.Bar(x=data.index,
                         y=data,
                         name=tech,
                         orientation='v',
                         marker_color=CMAP[i+1],
                         legendgroup=tech,
                         hovertemplate='%{y:.1f} t CO2',))
fig.update_layout(
    title='Emissions',
    barmode='stack',
    hovermode="x unified",
    #height=50*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_yaxes(title_text="t CO2")
fig.show() 

# Costs

## 6.2 LCOE, LCOH and RES share

In [ ]:
scenario_results = scenario_results[1].join(scenario_results[2].drop("Scenario", axis=1)).join(scenario_results[3].drop("Scenario", axis=1))

tmp_dict = {
    "RES share [%]": [],
    "Autark hours [%]": [],
    'Heat Storage Use [GWh]': [],
    'El. Storage Use [GWh]': [],
    'Net DSM activation [GWh]': []
}
for scenario in scenarios:
    tmp_dict["RES share [%]"].append(results_scns[scenario]["results_t"]['Electricity generation'].drop(
        ["Large-scale CHP", "Open-cycle gas turbine", "Combined-cycle gas turbine"]).sum() / float(results_scns[scenario]["highlevel_results"]["Electricity demand total"]) * 100)
    tmp_dict["Autark hours [%]"].append(float(results_scns[scenario]["highlevel_results"]["Autark hours"]) * 100)
    tmp_dict['Heat Storage Use [GWh]'].append(results_scns[scenario]["results_axlxt"]['Wärmespeicher nach Gemeinde'].sum().sum() / 1e3)
    tmp_dict['El. Storage Use [GWh]'].append(results_scns[scenario]["results_axlxt"]['Batteriespeicher nach Gemeinde'].sum().sum() / 1e3)
    tmp_dict['Net DSM activation [GWh]'].append(float(results_scns[scenario]["highlevel_results"]['Net DSM activation']) / 1e3)
additional_key_figures = pd.DataFrame(tmp_dict, index=scenarios)
scenario_results_ext = scenario_results.join(additional_key_figures, on="Scenario").set_index("Scenario").sort_values("RES share [%]")
scenario_results_ext = scenario_results_ext.drop("StatusQuo", axis=0)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

re_scn_matches = [re.search('((RE\+\+)|(RE\+)|(WIND\+)|(PV\+)|(RE-)|(RE))', scn) for scn in scenario_results_ext.index]
scenario_results_ext["RE scenario"] = [m[0].replace("WIND", "RE").replace("PV", "RE") if m is not None else "RE" for m in re_scn_matches]

re_scn_sort = {"RE-": 0, "RE": 1, "RE+": 2, "RE++": 3}
scenario_results_ext["RE scenario order"] = scenario_results_ext["RE scenario"].replace(re_scn_sort)
scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "RES share [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["RES share [%]"],
            x=xlabels,
            name="RES share in %",
            marker_color=CMAP[4],
            #hovertemplate='%{y:.2f}<extra></extra> %',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext["LCOE [EUR/MWh]"],
                             x=xlabels,
                             mode='markers',
                             opacity=0.7,
                             marker_color=CMAP[0],
                             name="LCOE in EUR/MWh",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext["LCOH [EUR/MWh]"],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="indianred",
                             name="LCOH in EUR/MWh",
                             ), secondary_y=True)


fig.update_layout(yaxis=dict(
                      title="RES share in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="LCOE/LCOH in EUR/MWh", secondary_y=True)
fig.update_xaxes()

# Flexibility

In [ ]:
scenario_results_ext["Flexibility shifted energy [GWh]"] = scenario_results_ext[["Heat Storage Use [GWh]", "El. Storage Use [GWh]","Net DSM activation [GWh]"]].sum(axis=1)

re_color_mapping = {"RE-": "mediumaquamarine", "RE": "lightseagreen", "RE+": "seagreen", "RE++": "yellowgreen"}
colors = [re_color_mapping[_] for _ in scenario_results_ext["RE scenario"]]

pairs = {
    "LCOE vs. Exports": ("LCOE [EUR/MWh]", "El. Exports [TWh]",),
    "Flexibility vs. Exports": ("Flexibility shifted energy [GWh]", "El. Exports [TWh]"),
    "Flexibility vs. Autarky": ("Flexibility shifted energy [GWh]", "Autarky [%]"),
    "LCOE vs. Autarky": ("LCOE [EUR/MWh]", "Autarky [%]",),
    "Flexibility vs. Intra-reg. Exchange": ("Flexibility shifted energy [GWh]", "Intra-reg. Exchange [TWh]"),
    "Autarky vs. Intra-reg. Exchange": ("Autarky [%]", "Intra-reg. Exchange [TWh]"),
    "Autark hours vs. Intra-reg. Exchange": ("Autark hours [%]", "Intra-reg. Exchange [TWh]"),
    "Flexibility vs. Emissions": ("Flexibility shifted energy [GWh]", "Specific Emissions [g/kWh]"),
}

for k, pair in pairs.items():
    fig = go.Figure(data=go.Scatter(
        x=scenario_results_ext[pair[1]],
        y=scenario_results_ext[pair[0]],
        mode='markers',
        marker=dict(size=[20] * len(scenarios),
                    color=colors),
        text=scenario_results_ext.index
    ))


    fig.update_yaxes(title_text=pair[0])
    fig.update_xaxes(title_text=pair[1])
    fig.show()

### 8.0.4 Flexibility utilization vs. Autarky

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "Autark hours [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["Autark hours [%]"],
            x=xlabels,
            name="Autark hours in %",
            marker_color=CMAP[4],
            #hovertemplate='%{y:.2f}<extra></extra> %',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext['El. Storage Use [%]'],
                             x=xlabels,
                             mode='markers',
                             opacity=0.7,
                             marker_color=CMAP[0],
                             name="Battery storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Heat Storage Use [%]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="indianred",
                             name="Heat storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['DSM Utilization Rate [%]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="darkorange",
                             name="DSM utilization in %",
                             ), secondary_y=True)


fig.update_layout(yaxis=dict(
                      title="Autark hours in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="Utilization in %", secondary_y=True)
fig.update_xaxes()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "RES share [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["RES share [%]"],
            x=xlabels,
            name="RES share in %",
            marker_color=CMAP[4],
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext['El. Storage Use [%]'],
                             x=xlabels,
                             mode='markers',
                             opacity=0.7,
                             marker_color=CMAP[0],
                             name="Battery storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Heat Storage Use [%]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="indianred",
                             name="Heat storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['DSM Utilization Rate [%]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="darkorange",
                             name="DSM utilization in %",
                             ), secondary_y=True)


fig.update_layout(yaxis=dict(
                      title="RES share in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="Utilization in %", secondary_y=True)
fig.update_xaxes()